# Manual feature extraction

# Imports

In [1]:
import os
import warnings

from attr.validators import instance_of
from lightgbm import LGBMClassifier
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
import lightgbm as lgb
from utils import variance_thresholding, standardize, calculate_metrics, calculate_metrics_statistics

In [2]:
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [12]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 60                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data"
DEPRESJON_PREFIX = "manual_depresjon"
PSYKOSE_PREFIX = "manual_psykose"
HYPERAKTIV_PREFIX = "manual_hyperaktiv"
MAIN_RESULTS_DIR = "results"

DAY_NIGHT_HOURS = (6, 22) # (6, 22) / (8, 21)
day_night_format = f'{DAY_NIGHT_HOURS[0]}_{DAY_NIGHT_HOURS[1]}' # "6, 22" / "8, 21"

# Classification

## Classifiers, parameters, constants

In [13]:
classifiers = {
    "GBM": LGBMClassifier(
    objective='binary',
    metric=['auc', 'binary_logloss'],
    force_col_wise=True,
    verbosity=-1
    ),
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=5000
    ),
    "SVM": SVC(
        kernel="poly",
        cache_size=512,
        max_iter=5000
    ),
    "RF": RandomForestClassifier(
        n_estimators=500,
        criterion="entropy"
    )
}


param_grids = {
    "GBM":{
    'num_leaves': [25, 50, 100, 250, 500],
    'max_depth': [5, 10, 15, 20, 25],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25],
    'reg_alpha': [0, 0.1, 0.2, 0.3],
    },
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": [None, "balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "C": np.logspace(10e-3, 10e3, num=50),
        "gamma": np.logspace(10e-3, 10e3, num=50),
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    }
}

## Hyperaktiv Classification

In [14]:
dataset = HYPERAKTIV_PREFIX
y_filename = f"hyperaktiv_{day_night_format}_y.csv"

In [15]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{day_night_format}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [16]:
from lightgbm_utils import lightgbm_fit_binary

results_directory = os.path.join(".", MAIN_RESULTS_DIR, "hyperactiv")

for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    
    
    for clf_type in ["GBM"]: 
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            if clf_type == "GBM":
                clf = lightgbm_fit_binary(X_train, y_train)
            else:
                grid_search = GridSearchCV(
                    estimator=classifiers[clf_type], 
                    param_grid=param_grids[clf_type], 
                    scoring="accuracy",
                    n_jobs=-1,
                    refit=True,
                    cv=LeaveOneOut()
                )
                grid_search.fit(X_train, y_train)
            
                clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"test_scores_{day_night_format}_{part}_{clf_type}.csv"), index=list(test_scores[0].keys()))
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"final_scores_{day_night_format}_{part}_{clf_type}.csv"), index=list(test_scores[0].keys()))
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        print()

PART: full_24h
  GBM


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.933634 + 0.358021


feature_fraction, val_score: 0.933634:  29%|##8       | 2/7 [00:00<00:00,  8.56it/s]

Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.933634 + 0.358021


...

[100]	cv_agg's valid binary_logloss: 1.76353 + 0.813163
Early stopping, best iteration is:
[10]	cv_agg's valid binary_logloss: 0.605605 + 0.144012
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.874864 + 0.297482


min_child_samples, val_score: 0.605605:  80%|########  | 4/5 [00:00<00:00, 10.54it/s]

Early stopping, best iteration is:
[28]	cv_agg's valid binary_logloss: 0.630506 + 0.1916
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.971773 + 0.299887


min_child_samples, val_score: 0.605605: 100%|##########| 5/5 [00:00<00:00, 11.81it/s]


{'accuracy': 0.8333333333333334, 'balanced_accuracy': np.float64(0.8116883116883117), 'f1': np.float64(0.7692307692307693), 'precision': np.float64(0.8333333333333334), 'recall': np.float64(0.7142857142857143), 'specificity': np.float64(0.9090909090909091), 'ROC_AUC': np.float64(0.8116883116883117), 'MCC': np.float64(0.6446583712203042)}
    accuracy: 0.8333 +- 0.0000
    balanced_accuracy: 0.8203 +- 0.0122
    f1: 0.7873 +- 0.0256
    precision: 0.8148 +- 0.0262
    recall: 0.7679 +- 0.0758
    specificity: 0.8727 +- 0.0514
    ROC_AUC: 0.8203 +- 0.0122
    MCC: 0.6534 +- 0.0123



## Depresjon classification

In [17]:
dataset = DEPRESJON_PREFIX
y_filename = f"depresjon_{day_night_format}_y.csv"

In [18]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{day_night_format}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [19]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, "depresjon")

for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    

    for clf_type in ["GBM"]: 
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            if clf_type == "GBM":
                clf = lightgbm_fit_binary(X_train, y_train)
            else:
                grid_search = GridSearchCV(
                    estimator=classifiers[clf_type], 
                    param_grid=param_grids[clf_type], 
                    scoring="accuracy",
                    n_jobs=-1,
                    refit=True,
                    cv=LeaveOneOut()
                )
                grid_search.fit(X_train, y_train)
            
                clf = grid_search.best_estimator_
                        
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"test_scores_{day_night_format}_{part}_{clf_type}"), index=list(test_scores[0].keys()))
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"final_scores_{day_night_format}_{part}_{clf_type}"), index=list(test_scores[0].keys()))
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

PART: full_24h
  GBM


feature_fraction, val_score: 0.971773:  14%|#4        | 1/7 [00:00<00:00, 15.28it/s]

Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.971773:  29%|##8       | 2/7 [00:00<00:00, 16.13it/s]

[100]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Training until validation scores don't improve for 100 rounds


...

min_child_samples, val_score: 0.796109:  40%|####      | 2/5 [00:00<00:00, 16.57it/s]

[100]	cv_agg's valid binary_logloss: 2.21784 + 1.62623
Early stopping, best iteration is:
[10]	cv_agg's valid binary_logloss: 0.796109 + 0.395334


min_child_samples, val_score: 0.796109: 100%|##########| 5/5 [00:00<00:00, 12.87it/s]

Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 1.25883 + 0.428606
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.970206 + 0.250319
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.972426 + 0.257546
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.972426 + 0.257546


{'accuracy': 0.5555555555555556, 'balanced_accuracy': np.float64(0.5844155844155844), 'f1': np.float64(0.5555555555555556), 'precision': np.float64(0.45454545454545453), 'recall': np.float64(0.7142857142857143), 'specificity': np.float64(0.45454545454545453), 'ROC_AUC': np.float64(0.5844155844155845), 'MCC': np.float64(0.16883116883116883)}
    accuracy: 0.6881 +- 0.1180
    balanced_accuracy: 0.6906 +- 0.0938
    f1: 0.6638 +- 0.0873
    precision: 0.6848 +- 0.2306
    recall: 0.6964 +- 0.0526
    specificity: 0.6848 +- 0.2306
    ROC_AUC: 0.6906 +- 0.0938
    MCC: 0.4065 +- 0.2208



## Psykose classification

In [20]:
dataset = PSYKOSE_PREFIX
y_filename = f"psykose_{day_night_format}_y.csv"

In [21]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{day_night_format}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [22]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, "psykose")

for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    
    for clf_type in ["GBM"]: 
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            if clf_type == "GBM":
                clf = lightgbm_fit_binary(X_train, y_train)
            else:
                grid_search = GridSearchCV(
                    estimator=classifiers[clf_type], 
                    param_grid=param_grids[clf_type], 
                    scoring="accuracy",
                    n_jobs=-1,
                    refit=True,
                    cv=LeaveOneOut()
                )
                grid_search.fit(X_train, y_train)
            
                clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"_test_scores_{day_night_format}_{part}_{clf_type}.csv"), index=False)
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"_final_scores_{day_night_format}_{part}_{clf_type}.csv"), index=False)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

PART: full_24h
  GBM


feature_fraction, val_score: 0.933634:  14%|#4        | 1/7 [00:00<00:00, 18.21it/s]

Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.933634:  43%|####2     | 3/7 [00:00<00:00, 16.83it/s]

[100]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.933634 + 0.358021
Training until validation scores don't improve for 100 rounds


...

[100]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Training until validation scores don't improve for 100 rounds


min_child_samples, val_score: 0.685492:  80%|########  | 4/5 [00:00<00:00,  9.35it/s]

[100]	cv_agg's valid binary_logloss: 1.70547 + 0.749582
Early stopping, best iteration is:
[13]	cv_agg's valid binary_logloss: 0.685492 + 0.213887
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.83269 + 0.4117
Early stopping, best iteration is:
[22]	cv_agg's valid binary_logloss: 0.697467 + 0.259239
Training until validation scores don't improve for 100 rounds
[100]	cv_agg's valid binary_logloss: 0.971773 + 0.299887
Early stopping, best iteration is:
[1]	cv_agg's valid binary_logloss: 0.971773 + 0.299887


min_child_samples, val_score: 0.685492: 100%|##########| 5/5 [00:00<00:00, 10.65it/s]


{'accuracy': 0.6666666666666666, 'balanced_accuracy': np.float64(0.6233766233766234), 'f1': np.float64(0.5), 'precision': np.float64(0.6), 'recall': np.float64(0.42857142857142855), 'specificity': np.float64(0.8181818181818182), 'ROC_AUC': np.float64(0.6233766233766234), 'MCC': np.float64(0.26856632724128343)}
    accuracy: 0.6852 +- 0.0262
    balanced_accuracy: 0.6689 +- 0.0400
    f1: 0.5972 +- 0.0708
    precision: 0.6167 +- 0.0118
    recall: 0.5893 +- 0.1193
    specificity: 0.7485 +- 0.0505
    ROC_AUC: 0.6689 +- 0.0400
    MCC: 0.3423 +- 0.0683



In [23]:
day_night_format

'6_22'